In [ ]:
!pip install streamlit fastapi uvicorn pyngrok plotly pandas numpy python-multipart

In [10]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import plotly.express as px

# -----------------------------
# PAGE CONFIG
# -----------------------------
st.set_page_config(page_title="Health Anomaly Dashboard", layout="centered")
st.title("🏥 Health Anomaly Detection Dashboard")

# -----------------------------
# FILE UPLOAD
# -----------------------------
uploaded_file = st.file_uploader(
    "Upload Fitness Dataset (CSV / JSON)",
    type=["csv", "json"]
)

@st.cache_data
def load_data(file):
    if file.name.endswith(".csv"):
        return pd.read_csv(file)
    else:
        return pd.read_json(file)

if uploaded_file:

    # -----------------------------
    # LOAD & PREPROCESS DATA
    # -----------------------------
    df = load_data(uploaded_file)

    required_columns = ['timestamp', 'heart_rate', 'sleep_tracking', 'step_count']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        st.error(f"Error: Missing required columns in the uploaded file: {', '.join(missing_columns)}. Please ensure your dataset contains these columns.")
        st.stop() # Stop further execution

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')

    features = ['heart_rate', 'sleep_tracking', 'step_count']
    df[features] = df[features].fillna(df[features].mean())

    st.success("✅ Data loaded and preprocessed successfully")

    # Display first five lines of the dataframe
    st.subheader('Raw Data Preview')
    st.dataframe(df.head())

    # -----------------------------
    # SIDEBAR FILTERS
    # -----------------------------
    st.sidebar.header("🔍 Filters")

    start_date = st.sidebar.date_input(
        "Start Date", df['timestamp'].min().date()
    )
    end_date = st.sidebar.date_input(
        "End Date", df['timestamp'].max().date()
    )

    mask = (
        (df['timestamp'].dt.date >= start_date) &
        (df['timestamp'].dt.date <= end_date)
    )
    df_filtered = df.loc[mask]

    # -----------------------------
    # ANOMALY DETECTION
    # -----------------------------
    model = IsolationForest(
        contamination=0.05,
        random_state=42
    )

    df_filtered['anomaly'] = model.fit_predict(
        df_filtered[features]
    )

    df_filtered['anomaly'] = df_filtered['anomaly'].map(
        {1: 0, -1: 1}
    )

    # -----------------------------
    # SUMMARY METRICS
    # -----------------------------
    st.subheader("📊 Anomaly Summary")
    st.metric("Total Records", len(df_filtered))
    st.metric("Total Anomalies", int(df_filtered['anomaly'].sum()))

    # -----------------------------
    # LINEAR VISUALIZATIONS
    # -----------------------------
    st.subheader("📈 Health Metric Trends")

    # HEART RATE GRAPH
    st.markdown("### ❤️ Heart Rate Trend")
    fig_hr = px.line(
        df_filtered,
        x='timestamp',
        y='heart_rate'
    )

    anomalies = df_filtered[df_filtered['anomaly'] == 1]
    fig_hr.add_scatter(
        x=anomalies['timestamp'],
        y=anomalies['heart_rate'],
        mode='markers',
        name='Anomaly'
    )

    st.plotly_chart(fig_hr, use_container_width=True)

    # SLEEP HOURS GRAPH
    st.markdown("### 😴 Sleep Duration Trend")
    fig_sleep = px.line(
        df_filtered,
        x='timestamp',
        y='sleep_tracking'
    )

    fig_sleep.add_scatter(
        x=anomalies['timestamp'],
        y=anomalies['sleep_tracking'],
        mode='markers',
        name='Anomaly'
    )

    st.plotly_chart(fig_sleep, use_container_width=True)

    # STEPS GRAPH
    st.markdown("### 🚶 Step Count Trend")
    fig_steps = px.line(
        df_filtered,
        x='timestamp',
        y='step_count'
    )

    fig_steps.add_scatter(
        x=anomalies['timestamp'],
        y=anomalies['step_count'],
        mode='markers',
        name='Anomaly'
    )

    st.plotly_chart(fig_steps, use_container_width=True)

    # -----------------------------
    # REPORT GENERATION
    # -----------------------------
    st.subheader("📄 Download Anomaly Report")

    report_df = df_filtered[df_filtered['anomaly'] == 1][
        ['timestamp', 'heart_rate', 'sleep_tracking', 'step_count']
    ]

    csv = report_df.to_csv(index=False)

    st.download_button(
        "⬇ Download CSV Report",
        data=csv,
        file_name="health_anomaly_report.csv",
        mime="text/csv"
    )

else:
    st.info("👆 Please upload a fitness dataset to start analysis")


Overwriting app.py


In [11]:
from pyngrok import ngrok

# Terminate any previous ngrok tunnels
ngrok.kill()

!nohup streamlit run app.py --server.port 80 --server.enableCORS false &>/dev/null &

# Open a ngrok tunnel to the Streamlit port
public_url = ngrok.connect(80)
print(f"Streamlit App URL: {public_url}")


Streamlit App URL: NgrokTunnel: "https://overtimorous-stirringly-eleni.ngrok-free.dev" -> "http://localhost:80"
